# <center> <b> <h1> Random Forest (Testing) </h1></b></center> 

<h1>Explanation about it: <h1>
    <p style="font-weight: normal; font-size: 18px; text-align: justify;"> 
        in this file the goal is evaluate the accuracy final with the test dataset
        that has not been used in al the process in order to avoid the overfitting 
    </p>
  

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import joblib
import pandas as pd

import numpy as np 

PATH_DATASETS = "../../2_Data_preparation/2_Split_Raw_Datasets/output/"
PATH_PKL = "../../3.5 Modeling/1_2_hyper_parameter_tunning/output/"
PATH_SIGNIFICANCE = "../../3 Modeling/2. Random Forest (significance Score)/output/"

In [2]:
def readCsvs(split):
    datasets = []
    constant1 = 'X_'
    constant2 = '_' + split
    features = ['area', 'meancurv', 'thickness', 'volume']

    for feature in features:
        df = pd.read_csv(PATH_DATASETS + constant1 + feature + constant2 +'.csv', index_col=0)
        datasets.append({'name': feature, 'data': df})
    return datasets

In [3]:
def readSignificances():
    df_number_of_features = pd.read_csv(PATH_SIGNIFICANCE 
                                        + "Random Forest Best Number Of Features.csv", index_col=0)
    
    datasets = []
    constant1 = 'Random Forest (AllFeatures - Significance - '
    constant2 = ')'
    features = ['area','meancurv', 'thickness', 'volume']
    number_of_feature = df_number_of_features['quantity'].values

    for feature, number in zip(features, number_of_feature):
        df = pd.read_csv(PATH_SIGNIFICANCE + constant1 + str(int(number)) + constant2 
                         + feature +'.csv', index_col=0)
        datasets.append({'name': feature[0], 'data': df})
    return datasets

In [4]:
def readClf():
    randomForests = []
    constant1 = "Random Forest (hyper parameter tuning)_"
    features = ['area', 'meancurv', 'thickness', 'volume']

    for feature in features:
        clf = joblib.load(PATH_PKL + constant1 + feature + '.pkl')
        randomForests.append({'name': feature, 'data': clf})
        
    return randomForests

# 1. Prepare the data

In [5]:
df_X_ts = readCsvs("test")
df_X_ts[0]['data'].shape

(220, 148)

In [6]:
df_y_ts = pd.read_csv(PATH_DATASETS + 'Y_test.csv', index_col = 0)
df_y_ts.shape

(220, 1)

In [7]:
Significances = readSignificances()
Significances[0]['data'].head()

,Significance
Name,
rh_G_and_S_cingul-Ant_area,0.020898
lh_S_temporal_sup_area,0.018332
rh_G_temporal_middle_area,0.017275
rh_S_temporal_sup_area,0.015924
lh_G_cuneus_area,0.015085


# 2. prepare the Clfs

In [8]:
clfs = readClf()

# 3. Testing

In [9]:
y_pred_vals = []
for feature, clf, significance in zip(df_X_ts, clfs, Significances):
    
    data = feature['data']
    name = feature['name']
    Rf = clf['data']
    
    y_pred_val = Rf.predict(data[significance['data'].index.values].copy())
    y_pred_vals.append({'name': name, 'data': y_pred_val})
    print("accuracy score test: " + name+'\n')
    print(accuracy_score(y_pred_val, df_y_ts['class'].values))

accuracy score test: area

0.759090909090909
accuracy score test: meancurv

0.7
accuracy score test: thickness

0.6772727272727272
accuracy score test: volume

0.7136363636363636


# feature importancce 

In [27]:
tables = []

for feature, clf, significance in zip(df_X_ts, clfs, Significances):
    
    data = feature['data'][significance['data'].index.values].copy()
    name = feature['name']
    Rf = clf['data']
    
    #not is a obb error is how much this variable apports to the model 
    df_features = pd.DataFrame(columns=['Name', 'Significance']) 
    df_features.set_index('Name', inplace = True)
    #df_features.head()

    for column in zip(data.columns.values, Rf.feature_importances_):
        #print(column)
        df_features.at[column[0]] = round(column[1],2)
    
    #print(df_features)
    #Total = df_features['Significance'].sum()
    #print ("for: " + name + " : " + str(Total))
    tables.append({'name' : name, 'data' : df_features})

    
#print(tables)

In [30]:
tables[3]['data'].sort_values('Significance', inplace = True, ascending=False)

In [33]:
tables[3]['data']

,Significance
Name,
lh_S_orbital_med-olfact_volume,0.1
rh_G_temporal_middle_volume,0.07
lh_G_temp_sup-Plan_polar_volume,0.06
lh_G_front_sup_volume,0.06
lh_Pole_occipital_volume,0.05
rh_S_temporal_inf_volume,0.05
rh_S_oc-temp_lat_volume,0.05
lh_G_rectus_volume,0.05
lh_G_temporal_middle_volume,0.05
